# Packages

In [1]:
# %pip -q install datasets
# %pip -q install evaluate


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [ ]:
import datasets
import evaluate
import numpy as np
from datasets import load_dataset
import pandas as pd

import transformers
from transformers import (
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
)

/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
2024-12-17 15:36:06.540714: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appr

In [3]:
from datasets.dataset_dict import DatasetDict
from datasets import Dataset

# Data

In [4]:
dev = '/home/jupyter/datasphere/project/rugec/data/RULEC-GEC.dev.tsv'
train = '/home/jupyter/datasphere/project/rugec/data/RULEC-GEC.train.tsv'
test = '/home/jupyter/datasphere/project/rugec/data/RULEC-GEC.test.tsv'

rulec_train = pd.read_csv(train, delimiter='\t')
rulec_test = pd.read_csv(test, delimiter='\t')
rulec_dev = pd.read_csv(dev, delimiter='\t')

In [5]:
rulec_dev.head()

,corrupt_sent,correct_sent
0,"Экскурсия прошла великолепно , Владимир Анатол...","Экскурсия прошла великолепно , Владимир Анатол..."
1,"Курсовая может быть о любой теме , которую обс...","Курсовая может быть по любой теме , которую об..."
2,"Я вижу , я слышу всё вокруг меня .","Я вижу , я слышу всё вокруг меня ."
3,Служить в Российской армии ( для мужчин ),Служить в Российской армии ( для мужчин )
4,Вряд ли все мир обеспокоился бы из-за бунта в ...,Вряд ли весь мир обеспокоился бы из-за бунта в...


In [6]:
fine_tune = {'train':Dataset.from_dict({'corrupt_sent':rulec_train['corrupt_sent'],'correct_sent' : rulec_train['correct_sent']}),
     'test':Dataset.from_dict({'corrupt_sent':rulec_test['corrupt_sent'],'correct_sent' : rulec_test['correct_sent']}),
      'dev':Dataset.from_dict({'corrupt_sent' : rulec_dev['corrupt_sent'], 'correct_sent':rulec_dev['correct_sent']})
     }

In [7]:
for each in fine_tune['train'].take(1):
    print(each)

{'corrupt_sent': 'В книге излагаются основы понимания обучения инностаного языка через разных теорий лингвистов в этом жанре .', 'correct_sent': 'В книге излагаются основы понимания обучения иностранному языку через разные теории лингвистов в этом жанре .'}


# Model and tokenization

In [ ]:
CONFIG = json.load(open('/home/jupyter/datasphere/project/tokens.json'))
tok = CONFIG["HF_TOK"]
wdb_tok = CONFIG['WANDB_API_KEY']

from huggingface_hub import login
import wandb

login(token=tok)
wandb.login(key=wdb_tok)

In [8]:
config = AutoConfig.from_pretrained('mika5883/pretrain_rugec_msu')
tokenizer = AutoTokenizer.from_pretrained('mika5883/pretrain_rugec_msu')
model = AutoModelForSeq2SeqLM.from_pretrained('mika5883/pretrain_rugec_msu')

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


In [9]:
def tokenize_function(examples):
    inputs = [f'grammar: {each}' for each in examples['corrupt_sent']]
    targets = [each for each in examples['correct_sent']]

    # inputs = [prefix + inp for inp in inputs] #we're skipping this step because our data is prefixed

    model_inputs = tokenizer(inputs, max_length=128, padding='max_length', truncation=True)
    labels = tokenizer(text_target=targets, max_length=128, padding='max_length', truncation=True)
    labels["input_ids"] = [
                [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
            ] #замена токенов паддинга на -100, чтобы они не учитывались при подсчёте потерь
    model_inputs["labels"] = labels["input_ids"]
    # model_inputs['corrupt_sent'] = inputs
    return model_inputs

In [10]:
train = fine_tune['train'].map(tokenize_function, batched=True)
test = fine_tune['test'].map(tokenize_function, batched=True)
dev = fine_tune['dev'].map(tokenize_function, batched=True)

Map: 100%|██████████| 2500/2500 [00:00<00:00, 3323.58 examples/s]


In [11]:
for each in train.take(1):
    print(each)

{'corrupt_sent': 'В книге излагаются основы понимания обучения инностаного языка через разных теорий лингвистов в этом жанре .', 'correct_sent': 'В книге излагаются основы понимания обучения иностранному языку через разные теории лингвистов в этом жанре .', 'input_ids': [8, 20849, 13555, 23, 32, 4219, 29, 2540, 310, 11753, 10227, 3887, 5, 8415, 1018, 99, 2855, 226, 1221, 30812, 8, 4819, 6468, 36, 6, 137, 7900, 13, 8, 4, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [13]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model='mika5883/pretrain_rugec_msu')

In [17]:
training_args = Seq2SeqTrainingArguments(
    output_dir="finetune_rugec_msu",
    eval_strategy="epoch",
    learning_rate=4e-5,
    per_device_train_batch_size=64,
    resume_from_checkpoint="last-checkpoint",
    per_device_eval_batch_size=64,
    weight_decay=0.18,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
    hub_strategy="checkpoint",
    report_to = 'all'
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=dev,
    tokenizer=tokenizer,
    data_collator=data_collator,
    # compute_metrics=compute_metrics,
)
transformers.logging.set_verbosity_info()
trainer.train()
# trainer.train(resume_from_checkpoint=False)

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: correct_sent, corrupt_sent. If correct_sent, corrupt_sent are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 4,980
  Num Epochs = 3
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 234
  Number of trainable parameters = 222,903,552
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Tracking run with wandb version 0.18.5
wandb: Run data is saved locally in /home/jupyte

{'eval_loss': 0.19952282309532166, 'eval_runtime': 6.3929, 'eval_samples_per_second': 391.056, 'eval_steps_per_second': 6.257, 'epoch': 1.0}



 67%|██████▋   | 156/234 [01:26<00:33,  2.30it/s]AThe following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: correct_sent, corrupt_sent. If correct_sent, corrupt_sent are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 2500
  Batch size = 64

 98%|█████████▊| 39/40 [00:06<00:00,  6.43it/s]
                                                 A
100%|██████████| 40/40 [00:06<00:00,  6.59it/s]
                                               

{'eval_loss': 0.19568416476249695, 'eval_runtime': 6.3726, 'eval_samples_per_second': 392.305, 'eval_steps_per_second': 6.277, 'epoch': 2.0}


100%|██████████| 234/234 [02:08<00:00,  2.30it/s]Saving model checkpoint to finetune_rugec_msu/checkpoint-234
Configuration saved in finetune_rugec_msu/checkpoint-234/config.json
Configuration saved in finetune_rugec_msu/checkpoint-234/generation_config.json
Model weights saved in finetune_rugec_msu/checkpoint-234/model.safetensors
tokenizer config file saved in finetune_rugec_msu/checkpoint-234/tokenizer_config.json
Special tokens file saved in finetune_rugec_msu/checkpoint-234/special_tokens_map.json
Copy vocab file to finetune_rugec_msu/checkpoint-234/spiece.model
tokenizer config file saved in finetune_rugec_msu/tokenizer_config.json
Special tokens file saved in finetune_rugec_msu/special_tokens_map.json
Copy vocab file to finetune_rugec_msu/spiece.model
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: correct_sent, corrupt_sent. If correct_sent, corrupt_sent are not expected by `T5ForCond

{'eval_loss': 0.19259853661060333, 'eval_runtime': 6.4272, 'eval_samples_per_second': 388.97, 'eval_steps_per_second': 6.224, 'epoch': 3.0}
{'train_runtime': 218.5302, 'train_samples_per_second': 68.366, 'train_steps_per_second': 1.071, 'train_loss': 0.20018367278270233, 'epoch': 3.0}


TrainOutput(global_step=234, training_loss=0.20018367278270233, metrics={'train_runtime': 218.5302, 'train_samples_per_second': 68.366, 'train_steps_per_second': 1.071, 'total_flos': 2274457721241600.0, 'train_loss': 0.20018367278270233, 'epoch': 3.0})

In [18]:
trainer.evaluate(test)

The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: correct_sent, corrupt_sent. If correct_sent, corrupt_sent are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 5000
  Batch size = 64
100%|██████████| 79/79 [00:12<00:00,  6.30it/s]


{'eval_loss': 0.2251938432455063,
 'eval_runtime': 12.6494,
 'eval_samples_per_second': 395.277,
 'eval_steps_per_second': 6.245,
 'epoch': 3.0}

In [19]:
trainer.push_to_hub('finetuned_msu')

Saving model checkpoint to finetune_rugec_msu
Configuration saved in finetune_rugec_msu/config.json
Configuration saved in finetune_rugec_msu/generation_config.json
Model weights saved in finetune_rugec_msu/model.safetensors
tokenizer config file saved in finetune_rugec_msu/tokenizer_config.json
Special tokens file saved in finetune_rugec_msu/special_tokens_map.json
Copy vocab file to finetune_rugec_msu/spiece.model
Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Sequence-to-sequence Language Modeling', 'type': 'text2text-generation'}}
events.out.tfevents.1734450211.l-6413647b-b198-4a9a-b7d3-3bd9d1634e63.2885.1: 100%|██████████| 359/359 [00:00<00:00, 1.72kB/s]


CommitInfo(commit_url='https://huggingface.co/mika5883/finetune_rugec_msu/commit/5beb8f7153022f66d39b3696c4092a5cd88533d6', commit_message='finetuned_msu', commit_description='', oid='5beb8f7153022f66d39b3696c4092a5cd88533d6', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mika5883/finetune_rugec_msu', endpoint='https://huggingface.co', repo_type='model', repo_id='mika5883/finetune_rugec_msu'), pr_revision=None, pr_num=None)